In [ ]:
import duckdb
import pandas as pd

# DataPrep

In [30]:
def dp_split_train_test(
    conn: duckdb.DuckDBPyConnection,
    tabela: str = 'abt_estacoes_3_vizinhas',
    min_medicoes: int = 10
) -> pd.DataFrame:
    """
    Cria uma coluna indicando o percentil temporal de cada medição dentro da sua estação.
    
    Parâmetros:
    -----------
    conn : duckdb.DuckDBPyConnection
        Conexão com o banco DuckDB
    tabela : str
        Nome da tabela a ser processada
    min_medicoes : int
        Número mínimo de medições para considerar uma estação
    
    Retorna:
    --------
    pd.DataFrame : DataFrame com a nova coluna percentil_temporal (valor entre 0 e 1)
    """
    
    query = f"""
    WITH estacoes_validas AS (
        -- Identifica estações com pelo menos {min_medicoes} medições
        SELECT id_estacao
        FROM {tabela}
        GROUP BY id_estacao
        HAVING COUNT(*) >= {min_medicoes}
    ),
    dados_ranqueados AS (
        -- Rankeia os registros de cada estação por data
        SELECT 
            t.*,
            ROW_NUMBER() OVER (PARTITION BY t.id_estacao ORDER BY t.dt_medicao) AS rn,
            COUNT(*) OVER (PARTITION BY t.id_estacao) AS total_registros
        FROM {tabela} t
        INNER JOIN estacoes_validas ev ON t.id_estacao = ev.id_estacao
    )
    SELECT 
        * EXCLUDE (rn, total_registros),
        CAST(rn AS DOUBLE) / total_registros AS percentil_temporal
    FROM dados_ranqueados
    ORDER BY id_estacao, dt_medicao;
    """
    
    df = conn.execute(query).df()
    
    return df

# Modelagem

In [ ]:
modelling_db = duckdb.connect('modelling_db')
final_db = duckdb.connect('final_db')
abt_table_name = 'abt_estacoes_3_vizinhas'
min_medicoes_estacao_valida=10

In [ ]:
df_split_train_test = dp_split_train_test(
        conn=modelling_db,
        tabela=abt_table_name,
        min_medicoes=min_medicoes_estacao_valida)